<a href="https://colab.research.google.com/github/gabrielletaree/UAS-Machine-Learning-Praktikum/blob/main/UASP_MLP_Gabriel_Letare_Napitupulu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Dependencies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from abc import ABC,abstractmethod
from typing import Dict, Any
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## **Pre-Processing Data**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Kuliah/MLP/trainingB.csv', names=(['wrist x', 'wrist y', 'wrist z', 'thigh x', 'thigh y', 'thigh z', 'label']))
test = pd.read_csv('/content/drive/MyDrive/Kuliah/MLP/SoalB.csv', names=(['wrist x', 'wrist y', 'wrist z', 'thigh x', 'thigh y', 'thigh z']))
df

,wrist x,wrist y,wrist z,thigh x,thigh y,thigh z,label
0,-0.468750,-1.156250,0.093750,-1.140625,0.109375,-0.562500,downstairs
1,-2.625000,0.281250,0.421875,-0.109375,0.125000,0.687500,jogging
2,-0.234375,-0.718750,0.093750,-1.984375,-0.046875,-0.625000,downstairs
3,-0.125000,-0.796875,0.093750,-0.734375,0.093750,-0.343750,downstairs
4,0.046875,-0.765625,0.015625,-0.640625,-0.125000,-0.468750,downstairs
...,...,...,...,...,...,...,...
23057,-0.312500,-1.109375,-0.109375,-0.843750,-0.390625,-0.046875,upstairs
23058,0.109375,-1.281250,0.187500,-2.609375,-1.062500,0.203125,jogging
23059,0.593750,-0.359375,-0.734375,-2.578125,-0.609375,-0.250000,jogging
23060,0.093750,0.578125,-0.031250,-0.562500,-0.703125,1.875000,jogging


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23062 entries, 0 to 23061
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   wrist x  23062 non-null  float64
 1   wrist y  23062 non-null  float64
 2   wrist z  23062 non-null  float64
 3   thigh x  23062 non-null  float64
 4   thigh y  23062 non-null  float64
 5   thigh z  23062 non-null  float64
 6   label    23062 non-null  object 
dtypes: float64(6), object(1)
memory usage: 1.2+ MB


In [ ]:
df.describe()

,wrist x,wrist y,wrist z,thigh x,thigh y,thigh z
count,23062.000000,23062.000000,23062.000000,23062.000000,23062.000000,23062.000000
mean,-0.478713,-0.521500,0.248979,-1.078784,-0.039724,-0.142223
std,0.736241,0.936527,0.716009,0.934523,0.564498,0.689882
min,-6.125000,-4.015625,-4.078125,-8.000000,-4.968750,-8.000000
25%,-0.609375,-1.156250,-0.109375,-1.437500,-0.250000,-0.375000
50%,-0.312500,-0.828125,0.031250,-0.937500,-0.015625,-0.156250
75%,-0.093750,0.250000,0.343750,-0.515625,0.218750,0.171875
max,4.421875,4.765625,4.187500,1.609375,4.937500,4.031250


In [ ]:
df['label'].value_counts()

jogging       9960
upstairs      6747
downstairs    6355
Name: label, dtype: int64

In [ ]:
def change_label(x):
  if 'jogging' in x:
    return 0
  if 'upstairs' in x:
    return 1
  else:
    return 2

df['label'] = df['label'].map(lambda x : change_label(x))

df = df.drop_duplicates()

In [ ]:
df

,wrist x,wrist y,wrist z,thigh x,thigh y,thigh z,label
0,-0.468750,-1.156250,0.093750,-1.140625,0.109375,-0.562500,2
1,-2.625000,0.281250,0.421875,-0.109375,0.125000,0.687500,0
2,-0.234375,-0.718750,0.093750,-1.984375,-0.046875,-0.625000,2
3,-0.125000,-0.796875,0.093750,-0.734375,0.093750,-0.343750,2
4,0.046875,-0.765625,0.015625,-0.640625,-0.125000,-0.468750,2
...,...,...,...,...,...,...,...
23057,-0.312500,-1.109375,-0.109375,-0.843750,-0.390625,-0.046875,1
23058,0.109375,-1.281250,0.187500,-2.609375,-1.062500,0.203125,0
23059,0.593750,-0.359375,-0.734375,-2.578125,-0.609375,-0.250000,0
23060,0.093750,0.578125,-0.031250,-0.562500,-0.703125,1.875000,0


In [ ]:
pd.concat([df[df.label == 0][:6355], df[df.label == 1][:6355], df[df.label == 2][:6355]])

,wrist x,wrist y,wrist z,thigh x,thigh y,thigh z,label
1,-2.625000,0.281250,0.421875,-0.109375,0.125000,0.687500,0
5,0.078125,0.859375,0.015625,-2.031250,-0.734375,-0.593750,0
9,-1.500000,0.734375,2.375000,-0.390625,-0.187500,0.000000,0
10,-1.046875,0.593750,-0.062500,-2.046875,0.796875,-1.359375,0
11,-0.687500,0.750000,-0.734375,-0.343750,-1.125000,1.078125,0
...,...,...,...,...,...,...,...
23037,0.265625,-0.968750,0.312500,-0.921875,-0.406250,-0.312500,2
23047,-0.250000,-0.718750,0.250000,-0.843750,-0.437500,-0.390625,2
23048,-0.390625,-1.000000,0.203125,-1.203125,-0.796875,-0.765625,2
23055,-0.406250,-0.687500,0.312500,-1.046875,0.000000,-0.125000,2


In [ ]:
test

,wrist x,wrist y,wrist z,thigh x,thigh y,thigh z
0,-0.296875,-0.703125,0.031250,-0.265625,-0.031250,0.000000
1,-0.171875,-1.140625,0.328125,-2.531250,-0.828125,0.203125
2,-0.343750,-1.031250,0.781250,-0.703125,0.500000,-0.125000
3,-0.234375,-0.671875,0.093750,-1.546875,-0.046875,-0.828125
4,0.265625,-1.000000,-0.062500,-1.875000,0.515625,0.281250
...,...,...,...,...,...,...
95,-0.656250,-1.812500,-0.031250,-1.140625,-0.234375,-0.296875
96,-0.296875,-1.093750,0.140625,-0.890625,0.250000,-0.546875
97,-0.468750,-1.453125,0.109375,-1.062500,-0.203125,-0.375000
98,-0.125000,-0.546875,-0.046875,-0.906250,0.031250,-0.343750


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   wrist x  100 non-null    float64
 1   wrist y  100 non-null    float64
 2   wrist z  100 non-null    float64
 3   thigh x  100 non-null    float64
 4   thigh y  100 non-null    float64
 5   thigh z  100 non-null    float64
dtypes: float64(6)
memory usage: 4.8 KB


## **Training**

In [ ]:
X_train = df.drop('label', axis=1)
y_train = df['label']

In [ ]:
X_test = test

In [ ]:
#Generating kNN Class
class KNN(ABC):
    """
    Base class for KNN implementations
    """
    def __init__(self, K: int = 3, metric: str = 'minkowski', p: int = 2) -> None:
        """
        Initializer function. Ensure that input parameters are compatiable.
        Inputs:
        K -> integer specifying number of neighbours to consider
        metric -> string to indicate the distance metric to use (valid entries are 'minkowski' or 'cosine')
        p -> order of the minkowski metric (valid only when distance == 'minkowski')
        """
        # check distance is a valid entry
        valid_distance = ['minkowski','cosine']
        if metric not in valid_distance:
            msg = "Entered value for metric is not valid. Pick one of {}".format(valid_distance)
            raise ValueError(msg)

        # check minkowski p parameter
        if (metric == 'minkowski') and (p <= 0):
            msg = "Entered value for p is not valid. For metric = 'minkowski', p >= 1"
            raise ValueError(msg)

        # store/initialise input parameters
        self.K = K
        self.metric = metric
        self.p = p
        self.X_train = np.array([])
        self.y_train = np.array([])

    def __del__(self) -> None:
        """
        Destructor function.
        """
        del self.K
        del self.metric
        del self.p
        del self.X_train
        del self.y_train

    def __minkowski(self, x: np.array) -> np.array:
        """
        Private function to compute the minkowski distance between point x and the training data X
        Inputs:
        x -> numpy data point of predictors to consider
        Outputs:
        np.array -> numpy array of the computed distances
        """
        return np.power(np.sum(np.power(np.abs(self.X_train - x),self.p),axis=1),1/self.p)

    def __cosine(self, x: np.array) -> np.array:
        """
        Private function to compute the cosine distance between point x and the training data X
        Inputs:
        x -> numpy data point of predictors to consider
        Outputs:
        np.array -> numpy array of the computed distances
        """
        return (1 - (np.dot(self.X_train,x)/(np.linalg.norm(x)*np.linalg.norm(self.X_train,axis=1))) )

    def __distances(self, X: np.array) -> np.array:
        """
        Private function to compute distances to each point x in X[x,:]
        Inputs:
        X -> numpy array of points [x]
        Outputs:
        D -> numpy array containing distances from x to all points in the training set.
        """
        # cover distance calculation
        if self.metric == 'minkowski':
            D = np.apply_along_axis(self.__minkowski,1,X)
        elif self.metric == 'cosine':
            D = np.apply_along_axis(self.__cosine,1,X)

        # return computed distances
        return D

    @abstractmethod
    def _generate_predictions(self, idx_neighbours: np.array) -> np.array:
        """
        Protected function to compute predictions from the K nearest neighbours
        """
        pass

    def fit(self, X: np.array, y: np.array) -> None:
        """
        Public training function for the class. It is assummed input X has been normalised.
        Inputs:
        X -> numpy array containing the predictor features
        y -> numpy array containing the labels associated with each value in X
        """
        # store training data
        self.X_train = np.copy(X)
        self.y_train = np.copy(y)

    def predict(self, X: np.array) -> np.array:
        """
        Public prediction function for the class.
        It is assummed input X has been normalised in the same fashion as the input to the training function
        Inputs:
        X -> numpy array containing the predictor features
        Outputs:
        y_pred -> numpy array containing the predicted labels
        """
        # ensure we have already trained the instance
        if (self.X_train.size == 0) or (self.y_train.size == 0):
            raise Exception('Model is not trained. Call fit before calling predict.')

        # compute distances
        D = self.__distances(X)

        # obtain indices for the K nearest neighbours
        idx_neighbours = D.argsort()[:,:self.K]

        # compute predictions
        y_pred = self._generate_predictions(idx_neighbours)

        # return results
        return y_pred

    def get_params(self, deep: bool = False) -> Dict:
        """
        Public function to return model parameters
        Inputs:
        deep -> boolean input parameter
        Outputs:
        Dict -> dictionary of stored class input parameters
        """
        return {'K':self.K,
                'metric':self.metric,
                'p':self.p}

#Generating kNN Classifier
class KNNClassifier(KNN):
    """
    Class for KNN classifiction implementation
    """
    def __init__(self, K: int = 3, metric: str = 'minkowski', p: int = 2) -> None:
        """
        Initializer function. Ensure that input parameters are compatiable.
        Inputs:
        K -> integer specifying number of neighbours to consider
        metric -> string to indicate the distance metric to use (valid entries are 'minkowski' or 'cosine')
        p -> order of the minkowski metric (valid only when distance == 'minkowski')
        """
        # call base class initialiser
        super().__init__(K,metric,p)

    def _generate_predictions(self, idx_neighbours: np.array) -> np.array:
        """
        Protected function to compute predictions from the K nearest neighbours
        Inputs:
        idx_neighbours -> indices of nearest neighbours
        Outputs:
        y_pred -> numpy array of prediction results
        """
        # compute the mode label for each submitted sample
        y_pred = stats.mode(self.y_train[idx_neighbours],axis=1).mode.flatten()

        # return result
        return y_pred

In [ ]:
knn = KNNClassifier(K=6)
knn.fit(X_train.values,y_train.values)
y_pred = knn.predict(X_test.values)
y_pred

array([1, 0, 2, 2, 0, 0, 2, 1, 0, 0, 0, 2, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       2, 0, 2, 2, 1, 1, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 1, 2, 0, 0, 2, 0,
       0, 2, 0, 2, 1, 0, 2, 2, 2, 0, 2, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2,
       1, 0, 0, 1, 2, 0, 0, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 2, 0, 1, 0, 0, 0, 1, 2, 1, 2, 0])

In [ ]:
len(y_pred)

100

In [ ]:
ID = list(range(1,101))

In [ ]:
df_dataset = pd.DataFrame({'ID' : ID ,'label': y_pred})

In [ ]:
df_dataset = df_dataset.astype({'label' : 'float'})
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      100 non-null    int64  
 1   label   100 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 1.7 KB


In [ ]:
label_mapping = {0: 'jogging', 1: 'upstairs', 2: 'downstairs'}
df_dataset['label'] = df_dataset['label'].replace(label_mapping)

df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      100 non-null    int64 
 1   label   100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [ ]:
df_dataset.to_csv('predictions.csv', index=False)

In [ ]:
df_dataset

,ID,label
0,1,upstairs
1,2,jogging
2,3,downstairs
3,4,downstairs
4,5,jogging
...,...,...
95,96,upstairs
96,97,downstairs
97,98,upstairs
98,99,downstairs
